# A toy example
# Convert to Julia file by jupytext --to jl ToyExample.ipynb

In [43]:
include("../src/DSI.jl")
include("../src/Zono_utils.jl")
include("../src/PZono.jl")
include("../src/DSZ.jl")

dsz_focal_refinement (generic function with 1 method)

In [44]:
print_figures = false;

In [45]:
# Toy example: defining the network
W1 = [1 -1.0; 1.0 1.]
b = [0.0; 0.0]
W2 = [1 -1.0; 1.0 1.]
L1 = Layer(W1, b, ReLU())
L2 = Layer(W2, b, Id())
full_net = Network([L1; L2])

# input range
x = [interval(-2,2),interval(-1.,1.)]

2-element Vector{Interval{Float64}}:
 [-2, 2]
 [-1, 1]

In [46]:
# Running the example with 2 focal elements
#nb_discretization_steps = 2
#ProbabilityBoundsAnalysis.setSteps(nb_discretization_steps)
#px = uniform.(inf.(x),sup.(x)) 
# Printing the pbox
#pz = pbox_approximate_nnet(full_net,px,true) # independent inputs
#if print_figures
#    ProbabilityBoundsAnalysis.plot(px[1])
#    ProbabilityBoundsAnalysis.plot(pz[1])
#    ProbabilityBoundsAnalysis.plot(pz[2])
#end

In [47]:
function Compare_analyzes(px::Vector{pbox})


    println("DSI analysis")
    @time dsi_pz = pbox_approximate_nnet(full_net,px,true)  # true for independent inputs
    print("dsi_pz[1] <= -2 : ",dsi_pz[1] <= -2,"\n")
    print("dsi_pz[2] >= 2 : ",dsi_pz[2] >= 2,"\n")

    
    println("Probabilistic zonotope analysis")
    @time pzono_pz = PZono_approximate_nnet(full_net,px,true) # true for independent inputs
    print("pzono_pz[1] <= -2 : ",pzono_pz[1] <= -2,"\n")
    print("pzono_pz[2] >= 2 : ",pzono_pz[2] >= 2,"\n")


    println("DSZ analysis")
    mat_spec_1 = [[1.0 0.0]
                  [0.0 -1.0]]
    rhs_spec_1 = [-2.0, -2.0]
    @time vec_proba = dsz_approximate_nnet_and_condition_nostorage(full_net, px, mat_spec_1,rhs_spec_1,true) 
    println("dsz_pz[1] <= -2 :", vec_proba[1])
    println("dsz_pz[2] >= 2 :", vec_proba[2])
    #println("dsz_pz[1] <= -2 and dsz_pz[2] >= 2 :", vec_proba[3])

end

Compare_analyzes (generic function with 1 method)

In [48]:
#ProbabilityBoundsAnalysis.setSteps(nb_discretization_steps)
#px = [(normal(interval(0,1),1)-0.5)/3.5902*2.,(normal(interval(0,1),1)/2.0-0.25)/1.7951]

println("Inputs from Uniform law (independent components)")
println("-----------------------")
for nb_discretization_steps in (2,10,100, 1000)
    println("Number of focal elements on the inputs = ",nb_discretization_steps)
    ProbabilityBoundsAnalysis.setSteps(nb_discretization_steps)
    px = uniform.(inf.(x),sup.(x)) 
    Compare_analyzes(px)
    println()
end

println("************************************************************************************************************************")
println()
println("Inputs from truncated Normal law (independent components)")
println("-------------------------------")
for nb_discretization_steps in (10,100, 1000)
    println("Number of focal elements on the inputs = ",nb_discretization_steps)
    px = init_pbox_Normal(inf.(x),sup.(x),nb_discretization_steps,true) # truncated normal law
    Compare_analyzes(px)
    println()
end

Inputs from Uniform law (independent components)
-----------------------
Number of focal elements on the inputs = 2
DSI analysis
  0.053183 seconds (36.79 k allocations: 1.812 MiB, 93.98% compilation time: 38% of which was recompilation)
dsi_pz[1] <= -2 : [0, 0.5]
dsi_pz[2] >= 2 : [0, 1]
Probabilistic zonotope analysis
  0.034802 seconds (29.80 k allocations: 1.433 MiB, 93.60% compilation time: 56% of which was recompilation)
pzono_pz[1] <= -2 : [0, 0.5]
pzono_pz[2] >= 2 : [0, 1]
DSZ analysis
  0.083902 seconds (94.11 k allocations: 4.703 MiB, 99.09% compilation time)
dsz_pz[1] <= -2 :[0, 0.25]
dsz_pz[2] >= 2 :[0, 0.5]

Number of focal elements on the inputs = 10
DSI analysis
  0.007280 seconds (18.40 k allocations: 872.172 KiB)
dsi_pz[1] <= -2 : [0, 0.2]
dsi_pz[2] >= 2 : [0, 0.7]
Probabilistic zonotope analysis
  0.004498 seconds (13.80 k allocations: 609.109 KiB)
pzono_pz[1] <= -2 : [0, 0.300001]
pzono_pz[2] >= 2 : [0, 0.800001]
DSZ analysis
  0.001302 seconds (6.88 k allocations: 37